In [ ]:
# Aula Prática nº4 - Agrupamento de Dados
# Grupo: Fabrício Fernandes Ziliotti - 11711BCC002
#	       Marcos Victor de Aquino Barra - 11711BCC007
#	       Salomão Oliveira Alves - 11711BCC038

In [1]:
#CELULA 01

import math
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [2]:
#CELULA 02

def write(clusters):
  arq = open('/content/gdrive/My Drive/UFU/Agrupamentos/Atividade10/cluster.txt', 'w')
  for cluster in clusters:
    arq.write(cluster)
    arq.write('\n')


def euclidiana(vetor1, vetor2):

	if(len(vetor1) != len(vetor2)):
		print("Os dois vetores devem possuir a mesma quantidade de elementos.")
		return

	qtdElementos, soma = len(vetor1), 0
	
	for i in range(qtdElementos):
		soma += math.pow(vetor1[i] - vetor2[i], 2)
  
	return math.sqrt(soma)
 

def calcula_dist(dataFrame):

  n_row = dataFrame.shape[0]
  matrix = []

  for i in range(n_row):
    vetor_exemplo = []

    for _ in range(i):
      vetor_exemplo.append(np.inf)

    for j in range(i, n_row):

      if i == j:
        vetor_exemplo.append(np.inf)
      else:
        diff = euclidiana(dataFrame.iloc[i,:], dataFrame.iloc[j,:])
        vetor_exemplo.append(diff)

    matrix.append(vetor_exemplo)

  return pd.DataFrame(matrix, columns = [str(i) for i in range(150)],
                              index = [str(i) for i in range(150)])


def recalcula(diff, idx1, idx2, cols):

  array = np.full((len(cols), len(cols)), np.inf)
  new_diff = pd.DataFrame(array, columns=cols, index=cols)
  cols_diff = diff.columns

  for i,col_i in enumerate(cols):
    for col_j in cols[i:]:

      # evita add valores na diag principal
      if col_i == col_j: 
        continue

      # novo grupo na coluna
      if col_j not in cols_diff and col_i in cols_diff: #col_i ta ok no diff
        if col_i < idx1:
          cel1 = diff.loc[col_i, idx1]
        else:
          cel1 = diff.loc[idx1, col_i]
        if col_i < idx2:
          cel2 = diff.loc[col_i, idx2]
        else:
          cel2 = diff.loc[idx2, col_i]
        new_diff.loc[col_i, col_j] = min(cel1, cel2)
        continue

      # novo grupo na linha
      if col_i not in cols_diff and col_j in cols_diff: #col_j ta ok no diff
        if col_j < idx1:
          cel1 = diff.loc[col_j, idx1]
        else:
          cel1 = diff.loc[idx1, col_j]
        if col_j < idx2:
          cel2 = diff.loc[col_j, idx2]
        else:
          cel2 = diff.loc[idx2, col_j]
        new_diff.loc[col_i, col_j] = min(cel1, cel2)
        continue

      new_diff.loc[col_i, col_j] = diff.loc[col_i, col_j]

  
  return new_diff

In [3]:
#CELULA 03

def main(diff):
  clusters = []

  while True:

    # pega os index do menor valor
    ind = np.unravel_index(np.argmin(diff, axis=None), diff.shape)
    idx1, idx2 = diff.columns[ind[0]], diff.columns[ind[1]]
    clusters.append(idx1+','+idx2)

    # define a coluna da nova matriz de distância (após o agrupamento)
    col = []
    for idx in diff.columns:
      if idx == idx1:
        col.append(idx1+','+idx2)
        continue
      if idx == idx2:
        continue
      col.append(idx)

    # para quando a qtd de grupos é 1
    if len(col) == 1:
      break
    
    # recalcula a nova matriz de distância
    diff = recalcula(diff, idx1, idx2, col)

  write(clusters) # gera o arq txt com os grupos

In [4]:
#CELULA 04

df = pd.read_csv('/content/gdrive/My Drive/UFU/Agrupamentos/Atividade10/iris.csv', header=None)
diff = calcula_dist(df)

main(diff)

In [ ]:
#CELULA 05

matrix = [[np.inf, 2.3, 3.4, 1.2, 3.7],
          [np.inf, np.inf, 2.6, 1.8, 4.6],
          [np.inf, np.inf, np.inf, 4.2, 0.7],
          [np.inf, np.inf, np.inf, np.inf, 4.4],
          [np.inf, np.inf,np.inf, np.inf,np.inf]]
diff = pd.DataFrame(matrix, columns = [str(i) for i in range(5)], index = [str(i) for i in range(5)])

main(diff)